# Objectif

Ce notebook d'introduction vous familiarise avec l'environnement du TP, avant les questions proprement dites.

Les données et les traitements se feront sur le cloud Google (Google Cloud Platform ou GCP), avec lequel nous interagirons grâce aux notebooks.

# I. L'interface BigQuery

Ouvrez une session de navigateur privée, et allez sur https://console.cloud.google.com.

Le login est de la forme `binome_xxx@thomasvial.fr`, `xxx` étant le n° de binôme (une lettre et 2 chiffres), et le mot de passe est composé de chiffres et de lettres en minuscules, fourni.

Après avoir accepté les conditions d’utilisation et quand la console s’affiche, sélectionner le projet `BUT-TP`.

Naviguer ensuite vers Big Query en tapant par exemple `big query` dans la barre de recherche. Vous devez arriver sur un écran qui ressemble au suivant :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01/refs/heads/main/TP3/images/bq.png)

Dans le panneau de gauche, vous devez voir les éléments de votre “explorateur” personnel. Il doit y avoir l’identifiant de votre binôme ; c’est un dataset BigQuery, qui permet d’organiser les objets créés (tables, etc.). Dans la capture d’écran, le dataset s’appelle `binome_zz`.

![](https://raw.githubusercontent.com/tvial/BUT-R6.01/refs/heads/main/TP3/images/explo.png)

Les 3 points en face du dataset permettent de créer une table avec sa structure, selon les options que vous donnerez dans les questions du TP.

### Vous pouvez commencer la partie 1 tu TP

# II. Interroger BigQuery depuis un notebook

## 1 - Chargement et configuration de l'extension

In [ ]:
%load_ext bigquery_magics

In [ ]:
import bigquery_magics
bigquery_magics.context.project = 'but-tp'

## 2 - Exécution d'une requête simple

Une requête est une cellule commençant par `%%bigquery`.

A la première requête, il faut s'authentifier à nouveau auprès de GCP. Il va nous demander d'entrer nos informations de connexion : mettez votre adresse de binôme (`binome_xxx@thomasvial.fr`), le mot de passe fourni, approuvez la connexion, et copiez le code affiché par la page avant de faire Entrée.

Si tout va bien, vous devriez voir le résultat, à savoir 3.

In [ ]:
%%bigquery
SELECT 1 + 2 as result

## 3 - Accès aux tables

Il y a une table `demo__mapping` dans le _dataset_ `shared` du projet. On peut la requêter comme suit. Notez la référence au dataset devant le nom de la table : c'est obligatoire.

In [ ]:
%%bigquery
SELECT * FROM shared.demo__mapping WHERE machine_id LIKE 'N%'

## 4 - Récupération dans un dataframe

En précisant un nom après `%%bigquery`, une variable Python sera créée. Ce sera un dataframe Pandas :

In [ ]:
%%bigquery machines_in_Naves
SELECT * FROM shared.demo__mapping WHERE machine_id LIKE 'N%'

In [ ]:
import re
machines_in_Naves['machine_name'].apply(lambda name: re.split('[-_ ]', name)[0]).value_counts()

## 5 - Création d'une table

Les tables que vous créerez seront stockées dans le dataset associé à votre binôme, qui porte son nom. Changez le nom du dataset dans la cellule ci-dessous avant de l'exécuter.

**Vous devrez référencer votre dataset à chaque fois que vous créerez ou lirez une table dedans.**

Notez également le `CREATE OR REPLACE TABLE`. Il permet de supprimer la table si elle existe déjà, ainsi on peut réexécuter la cellule plusieurs fois sans se poser de questions.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE binome_xxx.copy_of_mapping_Villemomble AS
  SELECT * from shared.demo__mapping
  WHERE machine_id LIKE 'V%'

In [ ]:
%%bigquery
SELECT * FROM binome_xxx.copy_of_mapping_Villemomble

### Vous pouvez commencer la partie 2 du TP, et revenir à ce notebook quand vous aurez besoin de conseils sur des points particuliers.

# III. Le dialecte SQL de BigQuery

BigQuery respecte en grande partie le standard SQL, mais il a aussi des extensions pour profiter de fonctionnalités additionnelles. Nous en passons quelques-unes en revue ici.

La documentation se trouve ici : [introduction au SQL de BigQuery](https://cloud.google.com/bigquery/docs/introduction-sql) et [référence des fonctions](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-all). **Consultez ces ressources en dernier recours, tenez plutôt compte des astuces ci-dessous qui vous feront gagner du temps, en évitant de chercher dans la documentation**.

## Transposition de tableaux

On a besoin de “déplier” en le considérant comme une mini-table, que l’on joint (produit cartésien) à la table originelle pour en démultiplier les lignes.

Le “dépliage” du tableau se fait avec la fonction `UNNEST()` de BigQuery, la jointure avec l’opérateur `CROSS JOIN` classique.

Voici un exemple simplifié, avec des vecteurs simples. Si notre table de départ, mytable, contient ceci :

In [ ]:
%%bigquery
SELECT * FROM shared.demo__unnest

On peut "déplier" ainsi :

In [ ]:
%%bigquery
SELECT id, element
FROM shared.demo__unnest
CROSS JOIN
UNNEST(myarray) AS element

Si on besoin de mettre en face la position de chaque élément dans le tableau, on peut étendre la requête :

In [ ]:
%%bigquery
SELECT id, element, position
FROM shared.demo__unnest
CROSS JOIN
UNNEST(myarray) AS element
WITH OFFSET AS position

## Création d'une UDF

Une UDF se fait avec l'ordre `CREATE [OR REPLACE] FUNCTION`. Il faut lui donner des paramètres, avec leur type, et préciser le type de retour. Le corps de la fonction, un ordre `SELECT` faisant intervenir les paramètres, se met ensuite après `AS` et est compris entre 2 jeux de parenthèses : `(( ... ))`.

Voici un exemple qui calcule la somme des éléments d'un vecteur :

In [ ]:
%%bigquery
CREATE OR REPLACE FUNCTION binome_xxx.vector_sum (vec ARRAY<FLOAT64>)
RETURNS FLOAT64
AS ((
    SELECT SUM(x)
    FROM UNNEST(vec) AS x
))

Notez le nom du type "flottant" dans BigQuery : `FLOAT64`, ainsi que l'utilisation d'un type "tableau" : `ARRAY<FLOAT64>`.

In [ ]:
%%bigquery
SELECT binome_xxx.vector_sum([1., 5., 9., -3.])

## Clauses `WITH`

Pour éviter d'écrire des requêtes trop compliquées, on les découpe en requêtes intermédiaires dont les résultats sont ensuite utilisées comme des tables virtuelles. C'est l'analogue de variables locales en programmation.

Exemple :

In [ ]:
%%bigquery

WITH mapping_with_machine_type AS (
    SELECT
        machine_name,
        SUBSTR(machine_id, 2, 1) AS machine_type
    FROM shared.demo__mapping
)
SELECT
    machine_type,
    COUNT(*) AS number
FROM mapping_with_machine_type
GROUP BY machine_type

Il est aussi possible d'enchaîner plusieurs clauses `WITH` qui se référencent les unes les autres :

In [ ]:
%%bigquery

WITH mapping_with_machine_type_and_site AS (
    SELECT
        machine_name,
        SUBSTR(machine_id, 2, 1) AS machine_type,
        SUBSTR(machine_id, 1, 1) AS site,
    FROM shared.demo__mapping
),
counts_per_type_and_site AS (
    SELECT
        machine_type,
        site,
        COUNT(*) AS number
    FROM mapping_with_machine_type_and_site
    GROUP BY machine_type, site
)
SELECT
    machine_type,
    SUM(number) AS total
FROM counts_per_type_and_site
GROUP BY machine_type

## Requête de pivotage

On fournit la requête toute faite :

In [ ]:
%%bigquery
WITH unfolded AS (
  SELECT
      vmd.* except (vector),
      element,
      'V' || CAST(position AS STRING) AS position
      FROM binome_xxx.vectors_with_metadata AS vmd
      CROSS JOIN UNNEST(vmd.vector) AS element WITH OFFSET position
)
SELECT *
FROM unfolded
PIVOT (
  MIN(element)
  FOR position IN (
    'V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15'
  )
)

Observer la structure du dataframe résultant.

Explication pour les curieux :

La pseudo-table `unfolded` est la version “dépliée” de `vectors_with_metadata` après éclatement des vecteurs eux-mêmes (au début, on avait juste déplié le niveau de tableau précédent). La position (`offset`) est transformée en chaîne de caractère `‘Vxx’` car ses valeurs détermineront le nom des colonnes du résultat.

Ensuite le résultat est pivoté, grâce à l’opérateur `PIVOT` ; il implique une agrégation, comme pour un tableau croisé dynamique. Ici les valeurs sont toutes uniques, l’opération `MIN()` n’a pas d’effet si ce n’est satisfaire le besoin d’agrégation.

## Entraînement d'un modèle de machine learning

Pour entraîner un modèle de K-means, il faut respecter le protocole suivant :

```
CREATE OR REPLACE MODEL binome_xxx.kmeans
TRANSFORM (...)
OPTIONS (
...
)
AS SELECT ...
```

Dans la partie `TRANSFORM`, on met entre parenthèses une liste de colonnes qui serviront de features pour l’entraînement. Ce peut être un sous-ensemble des colonnes du `SELECT`, ou des expressions dérivées de celles-ci. La transformation sera enregistrée avec le modèle et servira à l'inférence.

Les options sont des listes de type `PARAMETRE = valeur`, séparées par des virgules ; ce sont les hyperparamètres du modèle par exemple.

Le `SELECT` fournit le jeu de données d’entraînement. Dans ce cas précis, le plus simple est que cette clause soit de la forme `SELECT * FROM (requête pivot)`.

## Inférence

Pour l’inférence, la syntaxe est un peu différente et utilise la fonction `ML.PREDICT()` de BigQuery :

```
SELECT * 
FROM ML.PREDICT(
  MODEL binome_xxx.kmeans,
  (SELECT ...)
)
```

Le `SELECT` interne doit être entre parenthèse, et sa structure doit contenir les colonnes du `TRANSFORM` du jeu d’entraînement. Dans notre cas, ce `SELECT` doit donc reprendre la requête pivot, adaptée aux données d'inférence.